In [ ]:
from pathlib import Path
import pickle

In [ ]:
filename = "role_domain_1000_10000"

In [ ]:
with Path(filename).open("rb") as f:
    results = pickle.load(f)
len(results)

9000

In [ ]:
example = next(iter(results.values()))
example

{'type': 'succeeded',
 'content': '{\n  "function_definition": "def transform_data(data: pd.DataFrame, missing_values_threshold: float = 0.5) -> pd.DataFrame:",\n  "code": """\n    # Check for missing values\n    missing_values_count = data.isnull().sum()\n    total_values_count = data.shape[0]\n    missing_values_ratio = missing_values_count / total_values_count\n\n    # Drop columns with too many missing values\n    drop_columns = missing_values_ratio[missing_values_ratio > missing_values_threshold].index\n    data = data.drop(drop_columns, axis=1)\n\n    # Impute remaining missing values with the median\n    data = data.fillna(data.median())\n\n    # Normalize the data\n    scaler = StandardScaler()\n    data[data.columns] = scaler.fit_transform(data)\n\n    return data\n""",\n  "comment": "# Impute remaining missing values with the median",\n  "explanation": "The comment correctly describes the imputation step, but it doesn\'t mention that the data is also normalized after imputing

In [ ]:
fd_key = '"function_definition": '
code_key = '"code": '
com_key = '"comment": '
e_key = '"explanation": '
r_key = '"correct": '

check_keys = list(map(lambda x: x[1:-3], [fd_key, code_key, com_key, e_key, r_key]))

def prepare_data(example):
    if example["type"] != "succeeded":
        return None
    cont = example["content"]
    
    fd_s = cont.find(fd_key)
    code_s = cont.find(code_key)
    com_s = cont.find(com_key)
    e_s = cont.find(e_key)
    r_s = cont.find(r_key)

    fd_e = cont.find('",\n', fd_s) + 1
    code_e = cont.find('",\n', code_s) + 1
    com_e = cont.find('",\n', com_s) + 1
    e_e = cont.find('",\n', e_s) + 1
    r_e = cont.find('\n}', r_s)

    if not (fd_s < fd_e < code_s < code_e < com_s < com_e < e_s < e_e < r_s < r_e):
        return None

    splits = [cont[fd_s:fd_e], cont[code_s:code_e], cont[com_s:com_e], cont[e_s:e_e], cont[r_s:r_e]]
    prepared = {}
    for k_v in splits:
        split_p = k_v.find(":")
        k, v = k_v[:split_p][1:-1], k_v[split_p+1:].strip()
        # if not k in check_keys:
        #     return False
        assert k in check_keys

        if k == "correct":
            v = (v == "true")
        else:
            if v[:3] == '"""':
                v = v[3:]
            if v[-3:] == '"""':
                v = v[:-3]
            v = v.rstrip()
        prepared[k] = v

    return prepared

dataset = []
for v in results.values():
    prepared = prepare_data(v)
    if prepared:
        dataset.append(prepared)
        
print(len(dataset))
for k, v in dataset[-1].items():
    print(k)
    print(v)
    print()

8106
function_definition

def forecast_sales(sales_data: pd.DataFrame, forecast_horizon: int) -> pd.DataFrame:
    \"\"\"
    Forecast future sales using an ARIMA model.
    
    Args:
        sales_data (pd.DataFrame): DataFrame containing historical sales data.
        forecast_horizon (int): Number of periods to forecast.
    
    Returns:
        pd.DataFrame: DataFrame containing forecasted sales.
    \"\"\"

code

    sales_data = sales_data.set_index('date')
    
    # Fit an ARIMA model to the sales data
    model = ARIMA(sales_data['sales'], order=(1, 1, 1))
    model_fit = model.fit()
    
    # Generate forecasts for the desired horizon
    forecast = model_fit.forecast(steps=forecast_horizon)
    
    # Create a DataFrame with the forecasted sales
    forecast_df = pd.DataFrame({'date': pd.date_range(start=sales_data.index[-1] + pd.Timedelta(days=1), 
                                                     periods=forecast_horizon, 
                                            

In [ ]:
count = 0
for example in dataset:
    count += example["correct"]
print(count, len(dataset) - count)

4135 3971


In [ ]:
with Path("prepared_role_domain_1000_10000").open("wb") as f:
    pickle.dump(dataset, f)

In [ ]:
with Path("prepared_role_domain_1000_10000").open("rb") as f:
    print(pickle.load(f)[0])

{'function_definition': '"def transform_data(data: pd.DataFrame, missing_values_threshold: float = 0.5) -> pd.DataFrame:"', 'code': '\n    # Check for missing values\n    missing_values_count = data.isnull().sum()\n    total_values_count = data.shape[0]\n    missing_values_ratio = missing_values_count / total_values_count\n\n    # Drop columns with too many missing values\n    drop_columns = missing_values_ratio[missing_values_ratio > missing_values_threshold].index\n    data = data.drop(drop_columns, axis=1)\n\n    # Impute remaining missing values with the median\n    data = data.fillna(data.median())\n\n    # Normalize the data\n    scaler = StandardScaler()\n    data[data.columns] = scaler.fit_transform(data)\n\n    return data', 'comment': '"# Impute remaining missing values with the median"', 'explanation': '"The comment correctly describes the imputation step, but it doesn\'t mention that the data is also normalized after imputing the missing values."', 'correct': False}
